In [1]:
import pandas as pd
import numpy as np
import os
import string
import glob

pd.set_option('max_columns', 300)
pd.set_option('max_rows', 500)

# Day Hours & Productivity

Semi-structured Excel files are automatically emailed to `paul.washburn@majorbrands.com` on the 15th and last day of each month.  The data contains all hours for each worker in the warehouse.

**Note:  Negative records are filtered out of the data as they were not associated with valid `employee_id`s.  These records are few in number, but this must be understood from a high-level.**

## About the Data

The data for this report is sourced from the following:

- Summary data from ADP is emailed from HR bi-monthly (1st and 15th) to summarize all operations hours 
- A Roster is merged in to match managers and thus warehouse information
- In future, aggregated MTC1 data will be merged into this report to enable CPMH values 

Names are dropped from all data sets to protect employee privacy.  

In [2]:
from datetime import datetime as dt

base_dir = 'C:/users/pmwash/Desktop/Re-Engineered Reports/Day Hours/'

def drop_unnecessary_characters(str_list):
    str_list = [str(s).lower().replace(' ', '_') for s in str_list]
    str_list = [str(s).lower().replace('-_', '') for s in str_list]
    return str_list

def replace_unnamed_and_nans(col_list):
    new_col_list = list()
    for col in col_list:
        col = str(col)
        if '|nan' in col:
            newcol = col.replace('|nan', '')
            new_col_list.append(newcol)
        elif 'unnamed:_' in col:
            newcol = col.replace('unnamed:_', col_group)
            newcol = ''.join(c for c in newcol if not c.isdigit())
            new_col_list.append(newcol)
        else:
            new_col_list.append(col)
            col_group = col.split('|')[0]
    return new_col_list

def adjust_roster_id(roster_emp_id):
    fixed_id = [str(s)[:3] + str(s)[4:] for s in roster_emp_id]
    return fixed_id

def preprocess_hr_data(file_path):
    '''
    Accepts path to the export from ADP from HR which is emailed
    twice per month
    '''
    df = pd.read_csv(file_path, skiprows=8)
    
    # clean up column names
    df.loc[0] = col_specifier = drop_unnecessary_characters(df.loc[0])
    df.columns = drop_unnecessary_characters(df.columns)
    col_list = [a +'|'+ b for a,b in zip(df.columns, col_specifier)]
    df.columns = replace_unnamed_and_nans(col_list)
    df.drop(index=0, inplace=True)
    
    # set data types to numeric after removing miscellaneous symbols
    non_numeric_cols = ['labor_level_selected', 'employee_id', 'employee_name']
    numeric_cols = [col for col in df.columns if col not in non_numeric_cols]
    for col in numeric_cols:
        df[col] = df[col].str.replace('$', '')
        df[col] = df[col].str.replace(',', '')
        df[col] = df[col].str.replace('(', '-')
        df[col] = df[col].str.replace(')', '')
        df[col] = df[col].astype(np.float32)
        
    # capture date from the file name
    dat = file_path.split('Worked ')[1]
    df['starting_date'] = dt.strptime(dat.split(' - ')[0], '%m%d%Y')
    
    # set indices
    non_numeric_cols = ['starting_date'] + non_numeric_cols
    df.set_index(non_numeric_cols, inplace=True)
    df.index = df.index.droplevel('employee_name') #drop names for privacy
    
    # map in semantics for labor level
    labor_level_dict = {'/50/5220////' : 'Shipping Repacking',
                        '/50/6502////' : 'Shipping Wages',
                        '/50/6513////' : 'Shipping Casual', 
                        '/70/5220////' : 'Warehouse Repacking',
                        '/70/7202////' : 'Warehouse Wages',
                        '/70/7214////' : 'Warehouse Casual',
                        '/70/7201////' : 'Warehouse Management'}
    df['labor_level'] = df.index.get_level_values('labor_level_selected')
    df.labor_level = df.labor_level.map(labor_level_dict)
    df['month'] = dat_ix = df.index.get_level_values('starting_date')
    df.month = df.month.apply(lambda d: format(d, '%b'))
    df['pay_period'] = ['01' if str(d).split('-')[-1].split(' ')[0]=='01' else '02' for d in dat_ix]
    df['month_period'] = df.month.astype(str) + '_' + df.pay_period.astype(str)
    
    df.reset_index(inplace=True)
    df['employee_id'] = df['employee_id'].astype(str)
    df['employee_id'] = df['employee_id'].str.replace(' ', '').str.upper()
    df['employee_id'] = adjust_roster_id(df['employee_id'])
    
    df = df.loc[df['total|wages'] > 0]
    df['night_crew'] = df['labor_level'].apply(lambda s: 'Shipping' in str(s))
    
    return df

file_list = glob.glob(base_dir + '*.csv')
ops_hours_df = pd.DataFrame()
for file in file_list:
    ops_hours_df = ops_hours_df.append(preprocess_hr_data(file))

## Processed Dataset

Below are the first five rows of the dataset for your reference.  

In [15]:
def fetch_operations_roster(fpath):
    roster = pd.read_csv(fpath)
    roster.columns = drop_unnecessary_characters(roster.columns)
    roster.rename(columns={'position_id': 'employee_id'}, inplace=True)
    roster['employee_id'] = roster['employee_id'].astype(str)
    roster['employee_id'] = roster['employee_id'].str.replace(' ', '').str.upper()
    roster.drop(columns=['first_name', 'last_name'], inplace=True)
    return roster

def merge_roster_with_adp(roster_fpath, ops_hours_df, verbose=1):
    '''
    Combines ADP data with Roster from HR.
    '''
    # read in roster data
    roster_df = fetch_operations_roster(roster_fpath)

    # merge with ops_hours_df
    rows_before = ops_hours_df.shape[0]
    ops_df = ops_hours_df.merge(roster_df, on='employee_id', how='left')
    rows_after = ops_df.shape[0]

    # check what got dropped
    notinnewdata = ~ops_hours_df.employee_id.isin(ops_df.employee_id.tolist())
    dropped_from_data = ops_hours_df.loc[notinnewdata, 'employee_id'].unique().tolist()
    
    if verbose:
        print('Roster Columns:')
        print(roster_df.columns.tolist())
        print('''
        Merging in EMPLOYEE ROSTER data from HR.  

        This file needs to be updated by hand each month from ADAM COLEMAN or whoever is in that role. 

        Rows before merging in Roster:    {}
        Rows after merging in Roster:     {}

        The following Employee IDs were dropped in this process:
        {}
        '''.format(rows_before, rows_after, dropped_from_data))

    # add in which warehouse they are in
    manager_dict = {'Manning, Travis': 'STL', 
                    'nan': 'STL', 
                    'Hercher, Donald': 'STL', 
                    'Coffer, Wesley': 'KC',
                    'Surls, Kurtis': 'KC', 
                    'Ade, Richard': 'KC', 
                    'Jorgensen, Skylar': 'KC'}
    ops_df['warehouse'] = ops_df.reports_to_name.map(manager_dict)
    
    return ops_df

roster_fpath = base_dir + 'lookup data/operations_roster_03202018.csv'
ops_df = merge_roster_with_adp(roster_fpath, ops_hours_df, verbose=0)
ops_df.set_index(['warehouse', 'reports_to_name', 'employee_id', 'month_period']).head()

absence_no_pay|days  \
warehouse reports_to_name employee_id month_period                        
STL       Manning, Travis 3FT002657   Feb_01                        0.0   
                          3FT003097   Feb_01                        0.0   
                          3FT002821   Feb_01                        0.0   
                          3FT002844   Feb_01                        0.0   
                          3FT002970   Feb_01                        0.0   

                                                    absence_no_pay|hours  \
warehouse reports_to_name employee_id month_period                         
STL       Manning, Travis 3FT002657   Feb_01                         0.0   
                          3FT003097   Feb_01                         0.0   
                          3FT002821   Feb_01                         0.0   
                          3FT002844   Feb_01                         0.0   
                          3FT002970   Feb_01                         0.0   

                                                    absence_no_pay|money  \
warehouse reports_to_name employee_id month_period                         
STL       Manning, Travis 3FT002657   Feb_01                         0.0   
                          3FT003097   Feb_01                         0.0   
                          3FT002821   Feb_01                         0.0   
                          3FT002844   Feb_01                         0.0   
                          3FT002970   Feb_01                         0.0   

                                                    absence_no_pay|wages  \
warehouse reports_to_name employee_id month_period                         
STL       Manning, Travis 3FT002657   Feb_01                         0.0   
                          3FT003097   Feb_01                         0.0   
                          3FT002821   Feb_01                         0.0   
                          3FT002844   Feb_01                         0.0   
                          3FT002970   Feb_01                         0.0   

                                                    birthday|days  \
warehouse reports_to_name employee_id month_period                  
STL       Manning, Travis 3FT002657   Feb_01                  NaN   
                          3FT003097   Feb_01                  NaN   
                          3FT002821   Feb_01                  NaN   
                          3FT002844   Feb_01                  NaN   
                          3FT002970   Feb_01                  NaN   

                                                    birthday|hours  \
warehouse reports_to_name employee_id month_period                   
STL       Manning, Travis 3FT002657   Feb_01                   NaN   
                          3FT003097   Feb_01                   NaN   
                          3FT002821   Feb_01                   NaN   
                          3FT002844   Feb_01                   NaN   
                          3FT002970   Feb_01                   NaN   

                                                    birthday|money  \
warehouse reports_to_name employee_id month_period                   
STL       Manning, Travis 3FT002657   Feb_01                   NaN   
                          3FT003097   Feb_01                   NaN   
                          3FT002821   Feb_01                   NaN   
                          3FT002844   Feb_01                   NaN   
                          3FT002970   Feb_01                   NaN   

                                                    birthday|wages  \
warehouse reports_to_name employee_id month_period                   
STL       Manning, Travis 3FT002657   Feb_01                   NaN   
                          3FT003097   Feb_01                   NaN   
                          3FT002821   Feb_01                   NaN   
                          3FT002844   Feb_01                   NaN   
                          3FT002970   Feb

In [17]:
import plotly.offline as py
import plotly.graph_objs as go
import plotly.figure_factory as ff
py.init_notebook_mode(connected=True)

def plot_by_month(df, y, title, 
               seperate_y_axis=False, 
               x_axis_label='', y_axis_label='', 
               scale='linear', initial_hide=False,
               barmode='group'):
    '''
    Plot variables by month
    '''
    df = df.loc[:, y + ['month']].groupby('month')[y].sum()
    
    label_arr = list(df)
    series_arr = list(map(lambda col: df[col], label_arr))
    
    for col in df.columns:
        df[col] = df[col].apply(lambda x: round(x, 2))
    print(title); print('-' * 100); print(df.T)
    
    
    layout = go.Layout(
        barmode = barmode,
        title = title,
        legend = dict(orientation="h"),
        xaxis = dict(type='month',
                  title='Month'),
        yaxis=dict(
            title = y_axis_label,
            showticklabels = not seperate_y_axis,
            type = scale
        ),
        bargap=0.2
    )
    
    y_axis_config = dict(
        overlaying = 'y',
        showticklabels = False,
        type = scale )
    
    visibility = 'visible'
    if initial_hide:  visibility = 'legendonly'
        
    # make a trace for each series
    trace_arr = []
    for index, series in enumerate(series_arr):
        trace = go.Bar( #go.Scatter
            x=series.index, 
            y=series, 
            text=title, 
            name=label_arr[index],
            visible=visibility,
            opacity=0.7
        )
        # Add seperate axis for the series
        if seperate_y_axis:
            trace['yaxis'] = 'y{}'.format(index + 1)
            layout['yaxis{}'.format(index + 1)] = y_axis_config    
        trace_arr.append(trace)
    fig = go.Figure(data=trace_arr, layout=layout)
    py.iplot(fig)
    print('\n')

    
def interactive_bar_plot(ops_df, y_list, subgroup):
    for grp, df in ops_df.groupby(['warehouse', subgroup]):
        title = str(grp).replace('(', '').replace(')', '').replace('\'', '')
        plot_by_month(df, 
                      y=y_list, 
                      title=title, 
                      seperate_y_axis=False, 
                      y_axis_label='Total Wages ($)', 
                      scale='linear',
                      initial_hide=False)

# Total Employee Wages 

Below wages are plotted by various categories.  Numbers shown are summed by group and month to show changes over time.  

## Wages by Warehouse & Day/Night

`True` indicates "Night Crew", `False` indicates "Day Crew."

In [20]:
# specify parameters
subgroup = 'night_crew'
y_list = ['doubletime|wages', 'overtime|wages', 'regular|wages', 'total|wages']

print('''
Wages ($) by Warehouse & Day/Night Cre
''')

interactive_bar_plot(ops_df, y_list, subgroup)


Wages ($) by Warehouse & Day/Night Cre

KC, False
----------------------------------------------------------------------------------------------------
month                  Feb       Mar
doubletime|wages    241.49      0.00
overtime|wages    12148.85   2884.73
regular|wages     50051.11  27573.34
total|wages       62441.45  30458.08




KC, True
----------------------------------------------------------------------------------------------------
month                  Feb       Mar
doubletime|wages      0.00      0.00
overtime|wages     3927.32   5940.68
regular|wages     70242.31  35727.10
total|wages       77756.38  44206.66




STL, False
----------------------------------------------------------------------------------------------------
month                  Feb       Mar
doubletime|wages      0.00      0.00
overtime|wages     3049.35   2215.05
regular|wages     70071.01  38899.13
total|wages       78067.20  42978.86




STL, True
----------------------------------------------------------------------------------------------------
month                   Feb       Mar
doubletime|wages     540.88      0.00
overtime|wages       760.85   1280.91
regular|wages     120534.40  68296.88
total|wages       123491.50  71814.76


## Wages by Warehouse & Manager

In [18]:
# specify parameters
subgroup = 'reports_to_name'
y_list = ['doubletime|wages', 'overtime|wages', 'regular|wages', 'total|wages']

print('''
Wages ($) by Warehouse/Manager
''')

interactive_bar_plot(ops_df, y_list, subgroup)


Wages ($) by Warehouse/Manager

KC, Ade, Richard
----------------------------------------------------------------------------------------------------
month                 Feb      Mar
doubletime|wages     0.00     0.00
overtime|wages      58.25   195.66
regular|wages     6832.52  3563.11
total|wages       6890.76  3758.78




KC, Coffer, Wesley
----------------------------------------------------------------------------------------------------
month                  Feb       Mar
doubletime|wages      0.00      0.00
overtime|wages     3927.32   5940.68
regular|wages     70242.31  35727.10
total|wages       77756.38  44206.66




KC, Jorgensen, Skylar
----------------------------------------------------------------------------------------------------
month                 Feb     Mar
doubletime|wages     0.00     0.0
overtime|wages    1699.96   197.4
regular|wages     6615.70  3677.8
total|wages       8315.66  3875.2




KC, Surls, Kurtis
----------------------------------------------------------------------------------------------------
month                  Feb       Mar
doubletime|wages    241.49      0.00
overtime|wages    10390.64   2491.67
regular|wages     36602.89  20332.43
total|wages       47235.03  22824.10




STL, Hercher, Donald
----------------------------------------------------------------------------------------------------
month                  Feb       Mar
doubletime|wages    357.92      0.00
overtime|wages     2920.02   2144.17
regular|wages     72125.32  41091.64
total|wages       80350.70  44567.21




STL, Manning, Travis
----------------------------------------------------------------------------------------------------
month                   Feb       Mar
doubletime|wages     182.96      0.00
overtime|wages       890.18   1351.79
regular|wages     118480.09  66104.37
total|wages       121208.00  70226.41


## Wages by Warehouse & Labor Level

In [19]:
# specify parameters
subgroup = 'labor_level'
y_list = ['doubletime|wages', 'overtime|wages', 'regular|wages', 'total|wages']

print('''
Wages ($) by Warehouse/Labor Level
''')

interactive_bar_plot(ops_df, y_list, subgroup)


Wages ($) by Warehouse/Labor Level

KC, Shipping Casual
----------------------------------------------------------------------------------------------------
month                  Feb       Mar
doubletime|wages      0.00      0.00
overtime|wages      822.50   2275.39
regular|wages     29174.09  13495.24
total|wages       29996.59  15770.63




KC, Shipping Wages
----------------------------------------------------------------------------------------------------
month                  Feb       Mar
doubletime|wages      0.00      0.00
overtime|wages     3104.82   3665.29
regular|wages     41068.22  22231.86
total|wages       47759.79  28436.03




KC, Warehouse Casual
----------------------------------------------------------------------------------------------------
month                  Feb       Mar
doubletime|wages      0.00      0.00
overtime|wages     5716.52   1368.12
regular|wages     23243.36  13031.09
total|wages       28959.90  14399.21




KC, Warehouse Repacking
----------------------------------------------------------------------------------------------------
month                 Feb
doubletime|wages     0.00
overtime|wages     725.37
regular|wages      305.37
total|wages       1030.74




KC, Warehouse Wages
----------------------------------------------------------------------------------------------------
month                  Feb       Mar
doubletime|wages    241.49      0.00
overtime|wages     5706.96   1516.61
regular|wages     26502.38  14542.25
total|wages       32450.81  16058.87




STL, Shipping Casual
----------------------------------------------------------------------------------------------------
month                  Feb       Mar
doubletime|wages      0.00      0.00
overtime|wages      234.47    297.27
regular|wages     66455.78  38958.66
total|wages       66690.24  39255.93




STL, Shipping Repacking
----------------------------------------------------------------------------------------------------
month                Feb     Mar
doubletime|wages    0.00    0.00
overtime|wages      0.00    0.00
regular|wages     662.59  883.74
total|wages       662.59  883.74




STL, Shipping Wages
----------------------------------------------------------------------------------------------------
month                  Feb       Mar
doubletime|wages    540.88      0.00
overtime|wages      526.38    983.64
regular|wages     53416.03  28454.48
total|wages       56138.67  31675.09




STL, Warehouse Wages
----------------------------------------------------------------------------------------------------
month                  Feb       Mar
doubletime|wages      0.00      0.00
overtime|wages     3049.35   2215.05
regular|wages     70071.01  38899.13
total|wages       78067.20  42978.86


# Total Employee Hours 

The plots below reflect same plotting method as was employed above, only it is plotting employee hours on the y-axis.    

## Hours by Warehouse & Day/Night

`True` indicates "Night Crew", `False` indicates "Day Crew."

In [23]:
# specify parameters
subgroup = 'night_crew'
y_list = ['doubletime|hours', 'overtime|hours', 'regular|hours', 'total|hours']

print('''
Hours by Warehouse/Labor Level
''')

interactive_bar_plot(ops_df, y_list, subgroup)


Hours by Warehouse/Labor Level

KC, False
----------------------------------------------------------------------------------------------------
month                 Feb      Mar
doubletime|hours     5.73     0.00
overtime|hours     399.90    95.81
regular|hours     2450.58  1353.14
total|hours       2856.21  1448.95




KC, True
----------------------------------------------------------------------------------------------------
month                 Feb      Mar
doubletime|hours     0.00     0.00
overtime|hours     136.46   212.10
regular|hours     3780.26  1914.21
total|hours       4086.72  2246.31




STL, False
----------------------------------------------------------------------------------------------------
month                 Feb      Mar
doubletime|hours     0.00     0.00
overtime|hours      89.06    66.54
regular|hours     3091.73  1731.71
total|hours       3407.14  1906.25




STL, True
----------------------------------------------------------------------------------------------------
month                 Feb      Mar
doubletime|hours    12.00     0.00
overtime|hours      23.68    39.91
regular|hours     5838.86  3311.86
total|hours       5999.54  3462.27


## Hours by Warehouse & Manager

In [24]:
# specify parameters
subgroup = 'labor_level'
y_list = ['doubletime|hours', 'overtime|hours', 'regular|hours', 'total|hours']

print('''
Hours by Warehouse/Labor Level
''')

interactive_bar_plot(ops_df, y_list, subgroup)


Hours by Warehouse/Labor Level

KC, Shipping Casual
----------------------------------------------------------------------------------------------------
month                 Feb     Mar
doubletime|hours     0.00    0.00
overtime|hours      31.46   87.84
regular|hours     1711.19  789.81
total|hours       1742.65  877.65




KC, Shipping Wages
----------------------------------------------------------------------------------------------------
month                 Feb      Mar
doubletime|hours     0.00     0.00
overtime|hours     105.00   124.26
regular|hours     2069.07  1124.40
total|hours       2344.07  1368.66




KC, Warehouse Casual
----------------------------------------------------------------------------------------------------
month                 Feb     Mar
doubletime|hours     0.00    0.00
overtime|hours     197.18   48.52
regular|hours     1200.46  674.67
total|hours       1397.64  723.19




KC, Warehouse Repacking
----------------------------------------------------------------------------------------------------
month               Feb
doubletime|hours   0.00
overtime|hours    22.79
regular|hours     14.50
total|hours       37.29




KC, Warehouse Wages
----------------------------------------------------------------------------------------------------
month                 Feb     Mar
doubletime|hours     5.73    0.00
overtime|hours     179.93   47.29
regular|hours     1235.62  678.47
total|hours       1421.28  725.76




STL, Shipping Casual
----------------------------------------------------------------------------------------------------
month                 Feb      Mar
doubletime|hours     0.00     0.00
overtime|hours       8.08    10.12
regular|hours     3401.03  1981.56
total|hours       3409.11  1991.68




STL, Shipping Repacking
----------------------------------------------------------------------------------------------------
month               Feb   Mar
doubletime|hours   0.00   0.0
overtime|hours     0.00   0.0
regular|hours     33.89  45.6
total|hours       33.89  45.6




STL, Shipping Wages
----------------------------------------------------------------------------------------------------
month                 Feb      Mar
doubletime|hours    12.00     0.00
overtime|hours      15.60    29.79
regular|hours     2403.94  1284.70
total|hours       2556.54  1424.99




STL, Warehouse Wages
----------------------------------------------------------------------------------------------------
month                 Feb      Mar
doubletime|hours     0.00     0.00
overtime|hours      89.06    66.54
regular|hours     3091.73  1731.71
total|hours       3407.14  1906.25


# Abandoned Code Below.

In [8]:
# import matplotlib.pyplot as plt
# import seaborn as sns
# from math import ceil as roundup
# %matplotlib inline

# def plot_timeseries_by_category(ops_df, x, y='total|wages', category='labor_level', 
#                      sub_category='warehouse', suptitle=None, verbose=0):
#     '''
    
#     '''
#     all_categories = ops_df[category].unique()
#     n_levels = len(all_categories)
    
#     if verbose:
#         print('''
#         There are {} levels in this category:
#         {}
#         '''.format(n_levels, all_levels))

#     n_rows = roundup(n_levels/2)
#     fig, axes = plt.subplots(nrows=n_rows, ncols=2, figsize=(15, 6*n_rows), sharex=False)

#     for i, cat in enumerate(all_categories):
#         _df = ops_df.loc[ops_df[category] == cat]
#         _df =  _df.groupby([sub_category, x])[y].sum().reset_index(drop=False)

#         all_subcategories = _df[sub_category].unique()
        
#         for subcat in all_subcategories:
#             _df = _df.loc[_df[sub_category] == subcat]
#             N = np.arange(len(_df[x].unique()))
            
#             if _df.shape[0] == 0:
#                 pass
#             elif i < n_rows:
#                 _df.plot(x, y, kind='barh', ax=axes[i, 0])
#                 #axes[i, 0].set_xticks(N, _df[x])
#                 axes[i, 0].grid(alpha=.7)
#                 axes[i, 0].set_xlabel('Total Wages')
#                 axes[i, 0].set_ylabel('')
#                 axes[i, 0].set_title(subcat)
#                 axes[i, 0].legend(loc='best')
#             else:
#                 _df.plot(x, y, kind='barh', ax=axes[i-n_rows, 1])
#                 axes[i-n_rows, 1].grid(alpha=.7)
#                 axes[i-n_rows, 1].set_xlabel('Total Wages')
#                 axes[i-n_rows, 1].set_ylabel('')
#                 axes[i-n_rows, 1].set_title(subcat)
#                 axes[i-n_rows, 1].legend(loc='best')
#         sns.despine()
        
#     if suptitle != None: plt.suptitle(suptitle)
    
#     return None
        
# plot_by_category(ops_df, 
#                  x='month',
#                  y='total|wages', 
#                  category='labor_level', 
#                  sub_category='warehouse', 
#                  suptitle='Hours v. Wages by Manager')
